In [1]:
import pandas as pd

In [2]:
def prepare(data):
    data['import.start'] = pd.to_datetime(data['import.start'])
    data['import.stop'] = pd.to_datetime(data['import.stop'])
    data['created'] = pd.to_datetime(data['created'])
    data['started'] = pd.to_datetime(data['started'])
    data['finished'] = pd.to_datetime(data['finished'])
    
    data['starttime'] = (data['started'] - data['created']).dt.total_seconds() 
    data['dockertime'] = (data['finished'] - data['started']).dt.total_seconds() 
    data['importtime'] = (data['import.stop'] - data['import.start']).dt.total_seconds() 
    
    started_temp = pd.DataFrame()
    docker_temp = pd.DataFrame()
    import_temp = pd.DataFrame()
    
    for i in range(8,29,5):
        started_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+1]) - pd.to_datetime(data.iloc[:, i])).dt.total_seconds() , True)
        docker_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+2]) - pd.to_datetime(data.iloc[:, i+1])).dt.total_seconds() , True)
        import_temp.insert(range(8,29,5).index(i), "", (pd.to_datetime(data.iloc[:, i+4]) - pd.to_datetime(data.iloc[:, i+3])).dt.total_seconds() , True)
        
    
    started_temp.columns = ['starttime{}'.format(i) for i in range(1, 6)]
    docker_temp.columns = ['dockertime{}'.format(i) for i in range(1, 6)]
    import_temp.columns = ['importtime{}'.format(i) for i in range(1, 6)]
    
    data = pd.concat([data, started_temp, docker_temp, import_temp], axis=1)
    
    return data

In [3]:
def extract_imports(imp):
    imp = imp.split("'")
    l = ""
    
    for i in imp:
        if i == "(" or i == ")" or i == ",)" or i == "," or i == "()" or i == ", ":
            continue
        else:
            if l == "":
                l = i
            else:
                l = l + ";" + i
    return l

This is one experiment run with using the top 10 packages over all functions
The installationtime is set to minus -1 if there was no install as for native python packages as os

In [4]:
approach = "all_top_10"
col = ['imports', 'buldingtime', 'installtime', 'starttime', 'dockertime',
       'importtime', 'starttime1', 'starttime2', 'starttime3', 'starttime4',
       'starttime5', 'dockertime1', 'dockertime2', 'dockertime3',
       'dockertime4', 'dockertime5', 'importtime1', 'importtime2',
       'importtime3', 'importtime4', 'importtime5']

In [5]:
data = pd.read_csv("data/"+approach+"/run10.csv")
data = prepare(data)
imp = data[col[0]]
data = data[col[1:]]

In [6]:
for i in range(9,0,-1):
    tmp = pd.read_csv("data/"+approach+"/run0"+str(i)+".csv")
    tmp = prepare(tmp)
    tmp = tmp[col[1:]]
    data = (data + tmp)
data = data/10
data.insert(0, col[0],imp)

In [7]:
data.head()

,imports,buldingtime,installtime,starttime,dockertime,importtime,starttime1,starttime2,starttime3,starttime4,...,dockertime1,dockertime2,dockertime3,dockertime4,dockertime5,importtime1,importtime2,importtime3,importtime4,importtime5
0,argparse;copy;datetime;funcx;itertools;json;ma...,32.55,28.33,2.857089,4.959326,3.581705,0.800952,0.790513,0.768165,0.837131,...,1.551979,1.553525,1.547483,1.544654,1.555830,0.443851,0.448588,0.440111,0.437569,0.446751
1,argparse;copy;funcx;itertools;json;matplotlib;...,28.58,25.28,2.465303,4.592141,3.236354,0.652623,0.669909,0.637811,0.659890,...,1.550738,1.548923,1.543917,1.550800,1.546416,0.444314,0.443242,0.438259,0.445283,0.441971
2,argparse;sys,8.08,5.57,2.345542,1.310198,0.033489,0.745338,0.747160,0.842015,0.660901,...,1.036167,1.036344,1.036176,1.035654,1.036544,0.006620,0.006533,0.005598,0.006537,0.006139
3,CLEAN,8.18,5.67,2.429079,1.309476,0.006192,0.744059,0.750829,0.785918,0.762379,...,1.028581,1.027413,1.027378,1.027771,1.026981,0.000239,0.000153,0.000201,0.000240,0.000223
4,CLEAN;os,7.75,5.46,2.407829,1.296658,0.006051,0.684403,0.735868,0.708399,0.624067,...,1.028333,1.028070,1.027802,1.027639,1.027857,0.000296,0.000291,0.000304,0.000191,0.000234


This is the mapping between imports and function
For this merge and the following ones, there is no need to think about on which join-operation. I cleaned the data beforehand. That is, there will be always a pair 
```python
pd.merge(func_mapping,data, on='imports')
```

In [8]:
func_mapping = pd.read_csv('experiments/function_mapping.csv')
func_mapping = func_mapping.drop(2)
func_mapping['func4.imports'] = func_mapping[func_mapping['func4.imports'].str.len() >= 0]['func4.imports'].map(lambda x: extract_imports(x))
func_mapping = func_mapping[func_mapping['func4.imports'] != ""]
func_mapping.rename(columns={'func4.imports': 'imports'}, inplace=True)
func_mapping.drop(columns=['Unnamed: 0'], inplace=True)

In [9]:
func_mapping.head()

,func4.name,imports,Cluster
0,0-04-_check_slurm_job_statuses.py,subprocess,Job
3,873-04-call_mtrack.py,mtrack;sys,Data: mtrack
4,43-04-make_corr_plots.py,json;os;xpcs_webplot,Sci: Instruments
6,3-04-submit_slurm_job.py,os;subprocess,Job
7,397-01-rejection_sample.py,random,Trivial


Here is the mapping between functions and task
```python
pd.merge(func_mapping,task_mapping, on='func4.name')
```

In [10]:
task_mapping = pd.read_csv("funcx/task_function_mapping.csv")

In [11]:
task_mapping.head()

,task_uuid,func4.name,executiontime
0,000001d5-b3f0-4c17-8074-04f013f7bf52,0-04-_check_slurm_job_statuses.py,0.0243
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,0-04-_check_slurm_job_statuses.py,0.0331
2,00002642-f850-42b4-9e1e-2698c3396498,0-04-_check_slurm_job_statuses.py,0.0432
3,00002deb-cf79-43ed-8aff-3281938809fe,0-04-_check_slurm_job_statuses.py,0.0268
4,0000754b-898e-43dc-9b55-35d1a6fe6633,0-04-_check_slurm_job_statuses.py,0.0284


Here we have the timestamp for each task
For merging
```python
pd.merge(task_mapping,task_mapping, on='task_uuid')
```

In [12]:
task = pd.read_csv("funcx/tasks_pt1.csv")
task = pd.concat([task, pd.read_csv("funcx/tasks_pt2.csv")], ignore_index=True)
task['received'] = task['received'].interpolate()

In [13]:
task.head()

,task_uuid,user_id,endpoint_uuid,received
0,000001d5-b3f0-4c17-8074-04f013f7bf52,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.681417e+18
1,00002521-04ba-4fb6-9c38-92ba23dc00d7,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679386e+18
2,00002642-f850-42b4-9e1e-2698c3396498,286.0,cbc1f8d4-71b2-4a41-8ed4-9c75421eb89f,1.683246e+18
3,00002deb-cf79-43ed-8aff-3281938809fe,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.677897e+18
4,0000754b-898e-43dc-9b55-35d1a6fe6633,293.0,340e58fa-c5f5-46af-a7f6-81b8c36661de,1.679277e+18


Here are information about the endpoints
For merging:
```python
pd.merge(task,endpoints, on='endpoint_uuid')
```

In [14]:
endpoints = pd.read_csv("funcx/endpoints.csv")

In [15]:
endpoints.head()

,endpoint_uuid,X_type,ip,cluster
0,00000000-0000-4000-8000-000011110000,LocalProvider,216.80.70.252,6
1,001fecc2-1393-4bd9-950e-5645e15e3a53,SlurmProvider,141.142.140.194,10
2,006f558d-df82-45c2-b2d5-94274ef41a69,SlurmProvider,141.142.140.194,10
3,00824163-569c-4f0d-953d-097cd3b7179d,NaN,159.226.43.47,10
4,01a3324e-7ad9-4563-b598-0c2499068a39,LocalProvider,128.135.204.6,8


Example merging in the end:
```python
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
```

In [16]:
o = pd.merge(func_mapping,data, on='imports')
p = pd.merge(task_mapping,o, on='func4.name')
r = pd.merge(task, p, on='task_uuid')
r.to_csv("my_data.csv")

In [17]:
sorted_df = r.sort_values(by=["received"], ascending=True)
sorted_df.to_csv("my_data_sorted.csv")